# Analyze MLB Matchups

## Module Imports

In [118]:
import pandas as pd
from pandas import DataFrame
import sqlite3
from os import path
from datetime import date
from IPython.core.interactiveshell import InteractiveShell

## Global Variables

In [119]:
# Display all columns of DataFrames
pd.options.display.max_columns = None

# Print all output in a cell not just the last piece of output
InteractiveShell.ast_node_interactivity = "all"

# Location of sports data
DATA_DIR = 'C:\\Users\\Harry\\Documents\\LTCWFF\\ltcwff_files\\data'

# Open database connection
conn = sqlite3.connect(path.join(DATA_DIR, 'mlb.sqlite'))

## Helper Functions

In [120]:
def old_uncompress_table(table):
    rows = []

    for i in range(len(table.index)):
        game = table.iloc[i, :].to_dict()
        row_1 = { (key.split('Home ')[1] if 'Home' in key else key):(val) for (key, val) in game.items() if 'Away' not in key }
        row_2 = { (key.split('Away ')[1] if 'Away' in key else key):(val) for (key, val) in game.items() if 'Home' not in key }
        rows = rows + [row_1, row_2]

    df = pd.DataFrame(rows)
    df['Date'] = df['Date'].astype(str)
    df['Index'] = df[['Date', 'Team']].agg('_'.join, axis = 1)
    df = df.drop(['Date', 'Team'], axis = 1)
    df = df.set_index('Index')
    return df

In [121]:
def uncompress_table(table, team):
    rows = []

    for ind in table.index:
        game = table.loc[ind].to_dict()
        if table.loc[ind, 'Home Team'] == team:
            new_game = {}
            for i in range(len(table.columns)):
                if 2 <= i <= 38:
                    new_game[table.columns[i]] = game[table.columns[i + 1]] if i % 2 == 0 else game[table.columns[i - 1]]
                else:
                    new_game[table.columns[i]] = game[table.columns[i]]
            #table.loc[ind] = new_game
            rows.append(new_game)
        else:
            rows.append(game)

    df = DataFrame(rows)
    df.columns = list(table.columns[:2]) + [ ('' if i % 2 == 0 else 'Opponent ') + table.columns[i].split(' ')[-1] for i in range(2, 39) ] + list(table.columns[39:])
    return df


In [122]:
df = pd.read_sql(f'''SELECT * FROM new_games WHERE "Away Team" = "PHI" OR "Home Team" = "PHI"''', conn)
uncompress_table(df, 'PHI')

,game_id,Date,Team,Opponent Team,1,Opponent 1,2,Opponent 2,3,Opponent 3,4,Opponent 4,5,Opponent 5,6,Opponent 6,7,Opponent 7,8,Opponent 8,9,Opponent 9,10,Opponent 10,11,Opponent 11,12,Opponent 12,13,Opponent 13,R,Opponent R,H,Opponent H,E,Opponent E,SP,Opponent SP,WP,WP Record,LP,LP Record,SV,SV Count
0,PHI202104010,20210401,PHI,ATL,1,0,0,0,1,0,0,0,0,0,0,0,0,2,0.0,0.0,0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,None,None,3,2,9,7,1,1,Aaron Nola,Max Fried,1-0,1-0,Nate Jones,0-1,None,NaN
1,PHI202104030,20210403,PHI,ATL,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0.0,0.0,X,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None,4,0,8,1,0,0,Zack Wheeler,Charlie Morton,1-0,1-0,Charlie Morton,0-1,None,NaN
2,PHI202104040,20210404,PHI,ATL,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1.0,0.0,X,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None,2,1,8,4,0,1,Zach Eflin,Ian Anderson,1-0,1-0,Chris Martin,0-1,Hector Neris,1.0
3,PHI202104050,20210405,PHI,NYM,0,0,0,0,0,0,0,2,0,0,0,0,0,0,5.0,0.0,X,1.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None,5,3,7,10,1,1,Matt Moore,Jacob deGrom,2-0,2-0,Trevor May,0-1,Jose Alvarado,1.0
4,PHI202104060,20210406,PHI,NYM,0,0,0,0,0,0,1,2,0,0,0,0,1,4,0.0,0.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None,4,8,10,5,0,2,Chase Anderson,Marcus Stroman,1-0,1-0,Chase Anderson,0-1,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,PHI202107292,20210729,PHI,WAS,0,4,0,2,1,1,2,0,1,0,0,0,3,0,4.0,1.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,11,8,12,14,4,1,Cristopher Sanchez,Patrick Corbin,5-3,5-3,Sam Clay,0-4,None,NaN
101,PHI202107291,20210729,PHI,WAS,0,0,0,1,0,0,1,0,0,0,0,0,0,2,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,1,3,3,7,0,0,Zack Wheeler,Max Scherzer,8-4,8-4,Zack Wheeler,8-6,Kyle Finnegan,1.0
102,PIT202107300,20210730,PHI,PIT,0,3,0,2,0,0,0,0,0,2,0,0,0,0,0.0,0.0,0.0,X,NaN,NaN,NaN,NaN,NaN,NaN,None,None,0,7,1,12,0,0,Vince Velasquez,Wil Crowe,Wil Crowe,3-5,Vince Velasquez,3-6,None,NaN
103,PIT202107310,20210731,PHI,PIT,0,0,0,0,0,2,0,0,1,0,0,0,0,0,1.0,0.0,0.0,1,NaN,NaN,NaN,NaN,NaN,NaN,None,None,2,3,8,6,0,1,Aaron Nola,JT Brubaker,Chris Stratton,4-0,Jose Alvarado,6-1,None,NaN


## Odds Analysis

In [123]:
# Get today's date in our format
today = date.today()
datef = today.strftime("%Y%m%d")

# Get today's odds as a DataFrame from the database
odds = pd.read_sql(f'''SELECT * FROM odds WHERE date = {datef}''', conn)
odds = odds.set_index('game_id')
odds = odds.drop('Date', axis = 1)
odds

,Away Team,Home Team,Away Spread,Away Spread Odds,Home Spread,Home Spread Odds,Away ML,Home ML,Over,Over Odds,Under,Under Odds,Away 1H ML,Home 1H ML,Away 1H Spread,Away 1H Spread Odds,Home 1H Spread,Home 1H Spread Odds,1H Over,1H Over Odds,1H Under,1H Under Odds
game_id,,,,,,,,,,,,,,,,,,,,,,
WAS202108030,PHI,WAS,-1.5,-105,1.5,-114,-176,148,8.0,-112,8.0,-108,-210.0,168.0,-0.5,-142.0,0.5,116.0,4.5,104.0,4.5,-128.0
TOR202108030,CLE,TOR,1.5,-120,-1.5,100,176,-210,8.5,-115,8.5,-105,164.0,-205.0,0.5,110.0,-0.5,-134.0,4.5,-110.0,4.5,-110.0
DET202108030,BOS,DET,-1.5,100,1.5,-120,-148,126,10.0,-110,10.0,-110,-142.0,116.0,-0.5,-104.0,0.5,-118.0,5.5,-115.0,5.5,-105.0
TBA202108030,SEA,TBR,1.5,-142,-1.5,118,138,-164,8.0,-120,8.0,-102,124.0,-152.0,0.5,-114.0,-0.5,-106.0,4.5,-102.0,4.5,-120.0
CIN202108030,MIN,CIN,1.5,-196,-1.5,162,106,-124,9.0,-104,9.0,-118,104.0,-128.0,0.5,-140.0,-0.5,114.0,4.5,-120.0,4.5,-102.0
MIA202108030,NYM,MIA,-1.5,105,1.5,-126,-162,136,7.5,-118,7.5,-104,-150.0,122.0,-0.5,-105.0,0.5,-115.0,3.5,-144.0,3.5,118.0
TEX202108030,LAA,TEX,-1.5,125,1.5,-150,-132,112,8.5,-112,8.5,-108,-134.0,110.0,-0.5,106.0,0.5,-130.0,4.5,-128.0,4.5,104.0
MIL202108030,PIT,MIL,1.5,-108,-1.5,-111,188,-225,9.0,-105,9.0,-115,152.0,-188.0,1.5,-150.0,-1.5,122.0,5.5,130.0,5.5,-160.0
CHA202108030,KCR,CHW,1.5,106,-1.5,-128,210,-255,9.0,-105,9.0,-115,180.0,-225.0,1.5,-130.0,-1.5,106.0,4.5,-134.0,4.5,110.0


In [124]:
# Loop through each row of the odds DataFrame and append analysis for each game
for ind in odds.index:    
    home_team = odds.loc[ind, 'Home Team']
    away_team = odds.loc[ind, 'Away Team']
    
    # Get the player's table for that prop
    away = uncompress_table(pd.read_sql(f'''SELECT * FROM new_games WHERE "Away Team" = "{away_team}" OR "Home Team" = "{away_team}"''', conn), home_team)
    home = uncompress_table(pd.read_sql(f'''SELECT * FROM new_games WHERE "Away Team" = "{home_team}" OR "Home Team" = "{home_team}"''', conn), away_team)
    
    odds.loc[ind, 'Away Wins'] = len(away.loc[away['R'] > away['Opponent R']])
    odds.loc[ind, 'Away Losses'] = len(away.loc[away['R'] < away['Opponent R']])
    odds.loc[ind, 'Home Wins'] = len(home.loc[home['R'] > home['Opponent R']])
    odds.loc[ind, 'Home Losses'] = len(home.loc[home['R'] < home['Opponent R']])

    # Create average and cover columns
    #Wins	Losses	Average Spread	Median Margin	Covers	Not Covers	Pushes	Cover Pct	Average Total	Median Points	Overs	Unders	Total Pushes	Over Pct	1H Average Spread	1H Median Margin	1H Covers	1H Not Covers	1H Pushes	1H Cover Pct	1H Average Total	1H Median Points	1H Overs	1H Unders	1H Total Pushes	1H Over Pct	2H Average Spread	2H Median Margin	2H Covers	2H Not Covers	2H Pushes	2H Cover Pct	2H Average Total	2H Median Points	2H Overs	2H Unders	2H Total Pushes	2H Over Pct	1Q Average Spread	1Q Median Margin	1Q Covers	1Q Not Covers	1Q Pushes	1Q Cover Pct	1Q Average Total	1Q Median Points	1Q Overs	1Q Unders	1Q Total Pushes	1Q Over Pct	2Q Average Spread	2Q Median Margin	2Q Covers	2Q Not Covers	2Q Pushes	2Q Cover Pct	2Q Average Total	2Q Median Points	2Q Overs	2Q Unders	2Q Total Pushes	2Q Over Pct	3Q Average Spread	3Q Median Margin	3Q Covers	3Q Not Covers	3Q Pushes	3Q Cover Pct	3Q Average Total	3Q Median Points	3Q Overs	3Q Unders	3Q Total Pushes	3Q Over Pct	4Q Average Spread	4Q Median Margin	4Q Covers	4Q Not Covers	4Q Pushes	4Q Cover Pct	4Q Average Total	4Q Median Points	4Q Overs	4Q Unders	4Q Total Pushes	4Q Over Pct
    #odds.loc[ind, 'Away Wins'] = len(away.loc[((away['Home Team'] == away_team) & (away['Home R'] > away['Away R'])) | ((away['Away Team'] == away_team) & (away['Away R'] > away['Home R']))])
    #odds.loc[ind, 'Home Wins'] = len(home.loc[((home['Home Team'] == home_team) & (home['Home R'] > home['Away R'])) | ((home['Away Team'] == home_team) & (home['Away R'] > home['Home R']))])
    #odds.loc[ind, 'Away Losses'] = len(away.loc[((away['Home Team'] == away_team) & (away['Home R'] < away['Away R'])) | ((away['Away Team'] == away_team) & (away['Away R'] < away['Home R']))])
    #odds.loc[ind, 'Home Losses'] = len(home.loc[((home['Home Team'] == home_team) & (home['Home R'] < home['Away R'])) | ((home['Away Team'] == home_team) & (home['Away R'] < home['Home R']))])
    #odds.loc[ind, 'Away Median Margin'] = (sum(away.loc[away['Home Team'] == away_team]['Home R'] - away.loc[away['Home Team'] == away_team]['Away R']) + sum(away.loc[away['Away Team'] == away_team]['Away R'] - away.loc[away['Away Team'] == away_team]['Home R'])) / len(away)


#odds.loc[ind, 'Home Avg'] = table[prop_name].mean()
#odds.loc[ind, 'Finals Avg'] = table[-5:][prop_name].mean()
#odds.loc[ind, 'Playoff Cover %'] = (table[prop_name] > props_df.loc[ind, 'Over']).mean()
#odds.loc[ind, 'Finals Cover %'] = (table.iloc[-5:][prop_name] > props_df.loc[ind, 'Over']).mean()


In [125]:
odds

,Away Team,Home Team,Away Spread,Away Spread Odds,Home Spread,Home Spread Odds,Away ML,Home ML,Over,Over Odds,Under,Under Odds,Away 1H ML,Home 1H ML,Away 1H Spread,Away 1H Spread Odds,Home 1H Spread,Home 1H Spread Odds,1H Over,1H Over Odds,1H Under,1H Under Odds,Away Wins,Away Losses,Home Wins,Home Losses
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,
WAS202108030,PHI,WAS,-1.5,-105,1.5,-114,-176,148,8.0,-112,8.0,-108,-210.0,168.0,-0.5,-142.0,0.5,116.0,4.5,104.0,4.5,-128.0,41.0,64.0,45.0,60.0
TOR202108030,CLE,TOR,1.5,-120,-1.5,100,176,-210,8.5,-115,8.5,-105,164.0,-205.0,0.5,110.0,-0.5,-134.0,4.5,-110.0,4.5,-110.0,48.0,54.0,50.0,52.0
DET202108030,BOS,DET,-1.5,100,1.5,-120,-148,126,10.0,-110,10.0,-110,-142.0,116.0,-0.5,-104.0,0.5,-118.0,5.5,-115.0,5.5,-105.0,52.0,55.0,46.0,62.0
TBA202108030,SEA,TBR,1.5,-142,-1.5,118,138,-164,8.0,-120,8.0,-102,124.0,-152.0,0.5,-114.0,-0.5,-106.0,4.5,-102.0,4.5,-120.0,46.0,60.0,54.0,52.0
CIN202108030,MIN,CIN,1.5,-196,-1.5,162,106,-124,9.0,-104,9.0,-118,104.0,-128.0,0.5,-140.0,-0.5,114.0,4.5,-120.0,4.5,-102.0,50.0,56.0,56.0,50.0
MIA202108030,NYM,MIA,-1.5,105,1.5,-126,-162,136,7.5,-118,7.5,-104,-150.0,122.0,-0.5,-105.0,0.5,-115.0,3.5,-144.0,3.5,118.0,43.0,61.0,46.0,59.0
TEX202108030,LAA,TEX,-1.5,125,1.5,-150,-132,112,8.5,-112,8.5,-108,-134.0,110.0,-0.5,106.0,0.5,-130.0,4.5,-128.0,4.5,104.0,47.0,58.0,41.0,64.0
MIL202108030,PIT,MIL,1.5,-108,-1.5,-111,188,-225,9.0,-105,9.0,-115,152.0,-188.0,1.5,-150.0,-1.5,122.0,5.5,130.0,5.5,-160.0,49.0,56.0,53.0,53.0
CHA202108030,KCR,CHW,1.5,106,-1.5,-128,210,-255,9.0,-105,9.0,-115,180.0,-225.0,1.5,-130.0,-1.5,106.0,4.5,-134.0,4.5,110.0,42.0,62.0,42.0,64.0
